# Введение

<p>Построение композиции — важный подход в машинном обучении, который позволяет<br> объединять большое количество слабых алгоритмов в один сильный.<br> Данный подход широко используется на практике в самых разных задачах.</p>
<p>На лекциях был рассмотрен метод градиентного бустинга, который последовательно<br> строит композицию алгоритмов, причем каждый следующий алгоритм выбирается так,<br> чтобы исправлять ошибки уже имеющейся композиции. Обычно в качестве базовых<br> алгоритмов используют деревья небольшой глубины, поскольку их достаточно легко<br> строить, и при этом они дают нелинейные разделяющие поверхности.</p>
<p>Другой метод построения композиций — случайный лес. В нем, в отличие от<br> градиентного бустинга, отдельные деревья строятся независимо и без каких-либо<br> ограничений на глубину — дерево наращивается до тех пор, пока не покажет наилучшее качество на обучающей выборке.</p>
<p>В этом задании мы будем иметь дело с задачей классификации. В качестве функции потерь будем использовать log-loss</p>




In [4]:
import pandas as pd
import math
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

Загрузите выборку из файла gbm-data.csv с помощью pandas и преобразуйте ее в массив numpy (параметр values у
датафрейма). В первой колонке файла с данными записано, была или нет реакция. Все остальные колонки (d1 - d1776)
содержат различные характеристики молекулы, такие как размер, форма и т.д. Разбейте выборку на обучающую и тестовую,
используя функцию train_test_split с параметрами test_size = 0.8 и random_state = 241.


In [28]:
df = pd.read_csv('gbm-data.csv')
y = df['Activity'].values
X = df.loc[:, 'D1':'D1776'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=241)

Обучите GradientBoostingClassifier с параметрами n_estimators=250, verbose=True, random_state=241 и для каждого
значения learning_rate из списка [1, 0.5, 0.3, 0.2, 0.1] проделайте следующее:


In [29]:
def sigmoid(y_pred):
    # Преобразуйте полученное предсказание с помощью сигмоидной функции по формуле 1 / (1 + e^{−y_pred}),
    # где y_pred — предсказаное значение.
    return 1.0 / (1.0 + math.exp(-y_pred))

In [30]:
def log_loss_results(model, X, y):
    # Используйте метод staged_decision_function для предсказания качества на обучающей и тестовой выборке
    # на каждой итерации.
    results = []
    for pred in model.staged_decision_function(X):
        results.append(log_loss(y, [sigmoid(y_pred) for y_pred in pred]))

    return results

In [38]:
def plot_loss(learning_rate, test_loss, train_loss):
    # Вычислите и постройте график значений log-loss (которую можно посчитать с помощью функции
    # sklearn.metrics.log_loss) на обучающей и тестовой выборках, а также найдите минимальное значение метрики и
    # номер итерации, на которой оно достигается.
#     plt.figure()
#     plt.plot(test_loss, 'r', linewidth=2)
#     plt.plot(train_loss, 'g', linewidth=2)
#     plt.legend(['test', 'train'])
#     plt.savefig('plots/rate_' + str(learning_rate) + '.png')

    min_loss_value = min(test_loss)
    min_loss_index = test_loss.index(min_loss_value)
    return min_loss_value, min_loss_index

In [32]:
def model_test(learning_rate):
    model = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=250, verbose=True, random_state=241)
    model.fit(X_train, y_train)

    train_loss = log_loss_results(model, X_train, y_train)
    test_loss = log_loss_results(model, X_test, y_test)
    return plot_loss(learning_rate, test_loss, train_loss)

In [39]:
min_loss_results = {}
for learning_rate in [1, 0.5, 0.3, 0.2, 0.1]:
    min_loss_results[learning_rate] = model_test(learning_rate)

      Iter       Train Loss   Remaining Time 
         1           1.0190           18.17s
         2           0.9192           20.58s
         3           0.8272           17.70s
         4           0.7834           16.42s
         5           0.7109           15.43s
         6           0.6368           15.00s
         7           0.5797           14.51s
         8           0.5610           14.15s
         9           0.5185           13.81s
        10           0.4984           13.41s
        20           0.1999           12.37s
        30           0.1313           11.91s
        40           0.0790           11.42s
        50           0.0511           10.68s
        60           0.0352            9.97s
        70           0.0245            9.39s
        80           0.0162            8.80s
        90           0.0114            8.31s
       100           0.0077            7.73s
       200           0.0004            2.23s
      Iter       Train Loss   Remaining Time 
        

Как можно охарактеризовать график качества на тестовой выборке, начиная с некоторой итерации:переобучение
(overfitting) или недообучение (underfitting)? В ответе укажите одно из слов overfitting либо underfitting.


In [21]:
print('overfitting')

overfitting


приведите минимальное значение log-loss и номер итерации, на котором оно достигается, при learning_rate = 0.2.

In [40]:
min_loss_results

{0.1: (0.5269201872275844, 51),
 0.2: (0.531094637596885, 36),
 0.3: (0.5423141110024554, 10),
 0.5: (0.5582025523164261, 6),
 1: (0.5822942594278476, 0)}

In [42]:
min_loss_value, min_loss_index = min_loss_results[0.2]
print('{:0.2f} {}'.format(min_loss_value, min_loss_index))

0.53 36


На этих же данных обучите RandomForestClassifier с количеством деревьев, равным количеству итераций, на котором
достигается наилучшее качество у градиентного бустинга из предыдущего пункта, c random_state=241 и остальными
параметрами по умолчанию. Какое значение log-loss на тесте получается у этого случайного леса? (Не забывайте, что
предсказания нужно получать с помощью функции predict_proba)

In [44]:
model = RandomForestClassifier(n_estimators=min_loss_index, random_state=241)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]
test_loss = log_loss(y_test, y_pred)
print(test_loss)

0.5413812861804069
